In [55]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot  as plt

In [ ]:
img = cv2.imread("test_images/0b43460429.jpg")
print(img.shape)
plt.imshow(img, cmap='gray')

In [ ]:
faces_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_eye.xml")


faces  = faces_cascade.detectMultiScale(img, 1.3, 5)
faces

In [ ]:
for x, y, w, h in faces:
        face_img = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        eyes = eye_cascade.detectMultiScale(img)
        for (x2,y2,w2,h2) in eyes:
                face_img = cv2.rectangle(face_img, (x2, y2), (x2+w2, y2+h2), (0, 255, 0), 2)

plt.imshow(face_img)

In [ ]:
def get_cropped(img_path="test_images/0b43460429.jpg"):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
    faces = faces_cascade.detectMultiScale(gray, 1.3, 5)
    for x, y, w, h in faces:   
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]     
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color
    

In [ ]:
img = cv2.imread("test_images/0b43460429.jpg")
plt.imshow(img)

In [ ]:
cropped_img = get_cropped()
plt.imshow(cropped_img)

In [4]:
path_to_data = "./images_dataset/"
path_to_cr_data = "./images_dataset/cropped/"

In [ ]:
import os
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)
img_dirs

In [ ]:
import shutil
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [ ]:
cropped_image_dirs = []
celebrity_file_names_dict = {}

for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1]
    print(celebrity_name)
    
    celebrity_file_names_dict[celebrity_name] = []
    
    for entry in os.scandir(img_dir):
        roi_color = get_cropped(entry.path)
        if roi_color is not None:
            cropped_folder = path_to_cr_data + celebrity_name
            if not os.path.exists(cropped_folder):
                os.makedirs(cropped_folder)
                cropped_image_dirs.append(cropped_folder)
                print("Generating cropped images in folder: ",cropped_folder)
                
            cropped_file_name = celebrity_name + str(count) + ".png"
            cropped_file_path = cropped_folder + "/" + cropped_file_name 
            
            cv2.imwrite(cropped_file_path, roi_color)
            celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
            count += 1 

### wavelet transform

In [8]:
import numpy as np
import pywt
import cv2    

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [ ]:
im_har = w2d(cropped_img,'db1',5)
plt.imshow(im_har, cmap='gray')

In [20]:
y = []
X = []

celeb_dick = {}
for i, celeb in  enumerate(os.listdir("images_dataset/cropped")):
    celeb_dick[celeb] = i
print(celeb_dick)


for celeb in celeb_dick:
    for path in os.listdir(os.path.join(path_to_cr_data, celeb)):
        img = cv2.imread(os.path.join(path_to_cr_data, celeb, path))
        resize_raw = cv2.resize(img, (32, 32))
        wave_img = w2d(img, 'db1', 5)
        resize_wave = cv2.resize(wave_img, (32, 32))
        reshape_raw = resize_raw.reshape(32*32*3, 1)
        reshape_wave = resize_wave.reshape(32*32, 1)
        stacked = np.vstack((reshape_raw, reshape_wave))
        X.append(stacked)
        y.append(celeb_dick[celeb])
X = np.array(X).reshape(len(X), 4096)

{'lionel_messi': 0, 'maria_sharapova': 1, 'roger_federer': 2, 'serena_williams': 3, 'virat_kohli': 4}


In [21]:
X.shape

(187, 4096)

### lets train model

In [13]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import classification_report

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [44]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7446808510638298

In [46]:
print(clf.predict([X_test[1]]))
print(classification_report(y_test, clf.predict(X_test)))

[2]
              precision    recall  f1-score   support

           0       0.80      0.67      0.73        12
           1       0.75      0.86      0.80         7
           2       0.75      0.50      0.60         6
           3       0.75      1.00      0.86         9
           4       0.69      0.69      0.69        13

    accuracy                           0.74        47
   macro avg       0.75      0.74      0.74        47
weighted avg       0.75      0.74      0.74        47



## lets try different models

In [47]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [59]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto',probability=True),
        'params' : {
            'svc__C': [1,10,100,1000],
            'svc__kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear'),
        'params': {
            'logisticregression__C': [1,5,10]
        }
    }
}

In [ ]:
scores = []
best_estimater = {}
for algo, params in model_params.items():
    pipeline = make_pipeline(StandardScaler(), params['model'])
    grid = GridSearchCV(pipeline, params['params'], cv=5, return_train_score=False)
    grid.fit(X_train, y_train)
    scores.append({
        'model': algo,
        'best_score': grid.best_score_,
        'best_param': grid.best_params_
    })
    best_estimater[algo] = grid.best_estimator_
    df = pd.DataFrame(scores)
    print(df)

In [58]:
df

,model,best_score,best_param
0,svm,0.771429,"{'svc__C': 1, 'svc__kernel': 'linear'}"
1,random_forest,0.607143,{'randomforestclassifier__n_estimators': 10}
2,logistic_regression,0.778571,{'logisticregression__C': 1}


In [ ]:
best_estimater

{'svm': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('svc',
                  SVC(C=1, gamma='auto', kernel='linear', probability=True))]),
 'random_forest': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier',
                  RandomForestClassifier(n_estimators=10))]),
 'logistic_regression': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression',
                  LogisticRegression(C=1, solver='liblinear'))])}

In [63]:
best_estimater['svm'].score(X_test,y_test)

0.7872340425531915

In [64]:
best_estimater['random_forest'].score(X_test,y_test)

0.6170212765957447

In [65]:
best_estimater['logistic_regression'].score(X_test,y_test)

0.8085106382978723

In [66]:
best_clf = best_estimater['svm']

In [67]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, best_clf.predict(X_test))
cm

array([[11,  0,  0,  0,  1],
       [ 0,  5,  0,  2,  0],
       [ 0,  1,  3,  1,  1],
       [ 0,  0,  1,  8,  0],
       [ 0,  1,  1,  1, 10]])

In [ ]:
import joblib 
joblib.dump(best_clf, 'saved_model.pkl')

['saved_model.pkl']

In [69]:
import json
with open("class_dictionary.json","w") as f:
    f.write(json.dumps(celeb_dick))